In [65]:
from Bio.PDB import *
import pandas as pd
import warnings
from Bio import BiopythonWarning
warnings.simplefilter('ignore', BiopythonWarning)
import csv
import numpy as np
from Bio.PDB.Polypeptide import three_to_one, one_to_three                           
import math
import requests as r
from Bio import SeqIO
from io import StringIO

load affinity bench excel file

In [2]:
def load_pd(fName):
    df = pd.read_csv(fName)
    return df

abIn = "C:/Users/amber/Study_Documents/intern/inno/ppi with binding affinity/pdbbind/PDBbind_cn.csv"
ppi = load_pd(abIn)
ppi = np.array(ppi[1:])
print("Size of the data:", ppi.shape)
# print(ppi)


Size of the data: (2850, 27)


In [3]:
pid_set = []

for i in range(2850):
# for i in range(10):
#     print(ppi[i,:])
    ent_name = ppi[i,1]
#     .lower()
#   set of pid
    if ent_name not in pid_set:
        # there is binding affnity data with < or >
        if '=' in ppi[i,5]:
            # remove protein which binding affinity is not measured by kd
            #print(i)
            symbol = ppi[i,5].split('=')[0]
            #print(symbol)
            bindingaffinity = ppi[i,5].split('=')[1]
            # print(chaibindingaffinityn_list)
            if symbol == "Kd":
                pid_set.append(ent_name)
print(len(pid_set))

2615


There are 2615 protein pairs with Kd as affinity data

## method 1: using protein data bank

In [50]:
def download_all_pdb(pid_set):
   pdbl = PDBList()
   for elem in pid_set:
        print(elem)
        assert(len(elem) ==4)
        pdbl.retrieve_pdb_file(elem, pdir = './pdbbind/PDBs/', file_format='pdb')

download_all_pdb(pid_set)  

1a22
Structure exists: './pdbbind/PDBs/pdb1a22.ent' 
1a2k
Structure exists: './pdbbind/PDBs/pdb1a2k.ent' 
1a3b
Structure exists: './pdbbind/PDBs/pdb1a3b.ent' 
1acb
Structure exists: './pdbbind/PDBs/pdb1acb.ent' 
1ak4
Structure exists: './pdbbind/PDBs/pdb1ak4.ent' 
1akj
Structure exists: './pdbbind/PDBs/pdb1akj.ent' 
1atn
Structure exists: './pdbbind/PDBs/pdb1atn.ent' 
1avg
Structure exists: './pdbbind/PDBs/pdb1avg.ent' 
1avx
Structure exists: './pdbbind/PDBs/pdb1avx.ent' 
1avz
Structure exists: './pdbbind/PDBs/pdb1avz.ent' 
1axi
Structure exists: './pdbbind/PDBs/pdb1axi.ent' 
1ay7
Structure exists: './pdbbind/PDBs/pdb1ay7.ent' 
1b27
Structure exists: './pdbbind/PDBs/pdb1b27.ent' 
1b2s
Structure exists: './pdbbind/PDBs/pdb1b2s.ent' 
1b2u
Structure exists: './pdbbind/PDBs/pdb1b2u.ent' 
1b3s
Structure exists: './pdbbind/PDBs/pdb1b3s.ent' 
1b6c
Structure exists: './pdbbind/PDBs/pdb1b6c.ent' 
1bj1
Structure exists: './pdbbind/PDBs/pdb1bj1.ent' 
1bql
Structure exists: './pdbbind/PDBs/pdb1bql

In [52]:
if '3ohm' in pid_set:

    pdbl = PDBList()
    elem = '3ohm'
    pdbl.retrieve_pdb_file(elem, pdir = './pdbbind/PDBs/', file_format='pdb')
    pid_set.remove('3ohm') #position 1148
    pid_set.append('7sq2')
elem = '7sq2'
pdbl.retrieve_pdb_file(elem, pdir = './pdbbind/PDBs/', file_format='pdb')


Structure exists: './pdbbind/PDBs/pdb7sq2.ent' 


'./pdbbind/PDBs/pdb7sq2.ent'

In [58]:
if '4fqr' in pid_set:

    pdbl = PDBList()
    elem = '4fqr'
    pdbl.retrieve_pdb_file(elem, pdir = './pdbbind/PDBs/', file_format='pdb')

# large strcture, pdb file not able to download
pid_set.remove('4fqr')

Desired structure doesn't exists


In [60]:
# filter the pdb complex with only two chains and records chain id
def chain_info (ent_name, ent_list):
    parser = PDBParser()
    structure = parser.get_structure(ent_name, './pdbbind/PDBs/pdb'+ ent_name.lower() + '.ent') 
    chainnum = 0
    chainname = ""
    for chain in structure.get_chains():
        chainnum += 1
        chain_ID0 = chain.get_id()
        chainname += chain_ID0
    if chainnum == 2:
        ent_list[ent_name] = chainname
    return ent_list
    
ent_list = {}
for ele in pid_set:
    chain_info(ele, ent_list)
print(len(ent_list))

636


In [72]:
def get_seq(ent_name, chain_ID0, chain_ID1, seq_dict):
#     print(ent_names, chain_ID0, chain_ID1)
    parser = PDBParser()
    
    
    structure = parser.get_structure(ent_name, './pdbbind/PDBs/pdb'+ ent_name.lower() + '.ent')
    #     assert(len(structure) == 1)
    model = structure[0]

    w_name0 = ent_name+'_'+chain_ID0
    if w_name0 not in seq_dict:
        w_seq0 = ''

        chain0 = model[chain_ID0]
        ppb = PPBuilder()
        for pp in ppb.build_peptides(chain0):
            w_seq0 += pp.get_sequence()

        seq_dict[w_name0] = w_seq0
        
        
    w_name1 = ent_name+'_'+chain_ID1
    if w_name1 not in seq_dict:
        w_seq1 = ''

        chain1 = model[chain_ID1]
        ppb = PPBuilder()
        for pp in ppb.build_peptides(chain1):
            w_seq1 += pp.get_sequence()

        seq_dict[w_name1] = w_seq1
    return seq_dict

Using room termperatuer 298K to calculate dG

In [73]:
def get_dg(ent_name, chain_ID0, chain_ID1, kd, dg_info):
    
    w_name0 = ent_name +'_'+chain_ID0
    w_name1 =  ent_name +'_'+chain_ID1
    #print(ent_name)
    # transfer kd unit to M
    kdnum = kd[:-2]
    #print(kdnum)
    
    kdunit = kd[-2:]
    if kdunit == "nM":
        kdnum = float(kdnum) * 10 ** (-9)
    elif kdunit == "uM":
        kdnum = float(kdnum) * 10 ** (-6) 
    elif kdunit == "fM":
        kdnum = float(kdnum) * 10 ** (-15) 
    elif kdunit == "pM":
        kdnum = float(kdnum) * 10 ** (-12) 
    #print(kdnum)
    dg = (8.314/4184)*(float(298)) * math.log(float(kdnum))
   
    info = w_name0 + " " +  w_name1 + " " + str(dg)
    dg_info.append(info)
    return dg_info

In [75]:
seq_dict = {}
dg_dict ={}
dg_info = []

f1 = open("./pdbbind/pdbbind.seq.txt", "w")
f2 = open("./pdbbind/pdbbind.dg.txt", "w")

for i in range(2616):
# for i in range(10):
#     print(ppi[i,:])
    #print(i)
    ent_name = ppi[i,1][:4]
#     .lower()
    if ent_name in ent_list:
        chains = ent_list[ent_name]
        chain_list = list(chains)
        chain_ID0 = chain_list[0]
        chain_ID1 = chain_list[1]

        kd = ppi[i,5].split('=')[1]
        #print(i, ppi[i,], chain_ID0, chain_ID1, kd)
        
        get_seq(ent_name, chain_ID0, chain_ID1, seq_dict)
        get_dg(ent_name, chain_ID0, chain_ID1, kd, dg_info)
    
for elem in seq_dict:
    print(elem, seq_dict[elem], file=open("./pdbbind/pdbbind.seq.txt", "a"))

for elem in dg_info:
    print(elem, file=open("./pdbbind/pdbbind.dg.txt", "a"))

f1.close()
f2.close()

Only 95 protein pairs are recorded. Others are with multiple chains. 

One of example is 1BVK_DE:F, with chain D and E are Fv Hulys11 and Chain F is HEW lysozyme. see https://www.rcsb.org/3d-view/1BVK.


There exists missing case such as '1AK4'

https://www.rcsb.org/structure/1AK4

In [5]:
chain_seq = {}
parser = PDBParser()
ent_name = '1ak4'
structure = parser.get_structure(ent_name, './pdbbind/PDBs/pdb'+ ent_name.lower() + '.ent') 
model = structure[0]
chainnum = 0
for chain in structure.get_chains():
    chainnum += 1
    chain_ID = chain.get_id()
    print(chainnum)
    print(chain_ID)

    seq0 = ''
    chain0 = model[chain_ID]
    ppb = PPBuilder()
    for pp in ppb.build_peptides(chain0):
        seq0 += pp.get_sequence()
    print(seq0)
    chain_seq[chain_ID] = seq0


1
A
MVNPTVFFDIAVDGEPLGRVSFELFADKVPKTAENFRALSTGEKGFGYKGSCFHRIIPGFMCQGGDFTRHNGTGGKSIYGEKFEDENFILKHTGPGILSMANAGPNTNGSQFFICTAKTEWLDGKHVVFGKVKEGMNIVEAMERFGSRNGKTSKKITIADCGQLE
2
B
NPTVFFDIAVDGEPLGRVSFELFADKVPKTAENFRALSTGEKGFGYKGSCFHRIIPGFMCQGGDFTRHNGTGGKSIYGEKFEDENFILKHTGPGILSMANAGPNTNGSQFFICTAKTEWLDGKHVVFGKVKEGMNIVEAMERFGSRNGKTSKKITIADCGQLE
3
C
PIVQNLQGQMVHQAISPRTLNAWVKVVEEKAFSPEVIPMFSALSEGATPQDLNTMLNTVGGHQAAMQMLKETINEEAAEWDRLHPVHAGPIAPGQMREPRGSDIAGTTSTLQEQIGWMTHNPPIPVGEIYKRWIILGLNKIVRMY
4
D
PIVQNLQGQMVHQAISPRTLNAWVKVVEEKAFSPEVIPMFSALSEGATPQDLNTMLNTVGGHQAAMQMLKETINEEAAEWDRLHPVHAGPIAPGQMREPRGSDIAGTTSTLQEQIGWMTHNPPIPVGEIYKRWIILGLNKIVRMY


In [8]:
print(len(chain_seq['A']))
print(len(chain_seq['B']))
print(len(chain_seq['C']))
print(len(chain_seq['D']))

165
163
145
145


## method 2: using uniprot labels

In [4]:
def get_dg_uniprot(label1, label2, kd, dg_info_uniprot):
    
    kdnum = kd[:-2]
    #print(kdnum)
    
    kdunit = kd[-2:]
    if kdunit == "nM":
        kdnum = float(kdnum) * 10 ** (-9)
    elif kdunit == "uM":
        kdnum = float(kdnum) * 10 ** (-6) 
    elif kdunit == "fM":
        kdnum = float(kdnum) * 10 ** (-15) 
    elif kdunit == "pM":
        kdnum = float(kdnum) * 10 ** (-12) 
    #print(kdnum)
    dg = (8.314/4184)*(float(298)) * math.log(float(kdnum))
   
    info = label1 + " " +  label2 + " " + str(dg)
    dg_info_uniprot.append(info)
    return dg_info_uniprot

In [31]:
uniprot_in = "C:/Users/amber/Study_Documents/intern/inno/ppi with binding affinity/uniprot.csv"
uniprot_seq = load_pd(uniprot_in)
print(uniprot_seq.shape)


(566996, 2)


extract sequencing from uniprot lib (the data in uniprot.csv is not complete)

In [81]:
def seq_uniprot_lib (label):
    cID=label

    baseUrl="http://www.uniprot.org/uniprot/"
    currentUrl=baseUrl+cID+".fasta"
    response = r.post(currentUrl)
    cData=''.join(response.text)

    Seq=StringIO(cData)
    pSeq=list(SeqIO.parse(Seq,'fasta'))
    result =str(pSeq[0].seq)
    return result

seq_uniprot_lib('Q07011')


'MGNSCYNIVATLLLVLNFERTRSLQDPCSNCPAGTFCDNNRNQICSPCPPNSFSSAGGQRTCDICRQCKGVFRTRKECSSTSNAECDCTPGFHCLGAGCSMCEQDCKQGQELTKKGCKDCCFGTFNDQKRGICRPWTNCSLDGKSVLVNGTKERDVVCGPSPADLSPGASSVTPPAPAREPGHSPQIISFFLALTSTALLFLLFFLTLRFSVVKRGRKKLLYIFKQPFMRPVQTTQEEDGCSCRFPEEEEGGCEL'

In [91]:
def get_seq_uniprot (label,seq_uniprot):
    
    data = uniprot_seq.loc[uniprot_seq['uniprot_id'] == label,'pro_seq']
    if data.empty:
        seq_from_lib = seq_uniprot_lib(label)
        seq_data = label + ' ' + seq_from_lib
        seq_uniprot.append(seq_data)
    else:
        seq = data.values[0]
        #print(seq)
        seq_data = label + ' ' + seq
        seq_uniprot.append(seq_data)
        #print(seq_uniprot)
    



In [94]:
f1 = open("./pdbbind/pdbbind_uniprot.seq.txt", "w")
f2 = open("./pdbbind/pdbbind_uniprot.dg.txt", "w")
dg_info_uniprot = []
label_list = []
seq_uniprot = []


for i in range(2850):
# for i in range(10):
#     print(ppi[i,:])
    ent_name = ppi[i,1]
#     .lower()

    if ent_name in pid_set:
        uniprot_label  = ppi[i,14]
        if type(uniprot_label) == str:
            uniprot_label_list = list(uniprot_label.split(' '))
            if len(uniprot_label_list) == 2:
                label1 = uniprot_label_list[0]
                label2 = uniprot_label_list[1]
                kd = ppi[i,5].split('=')[1]

                if label1 not in label_list:
                    label_list.append(label1)
                    get_seq_uniprot(label1, seq_uniprot)
                if label2 not in label_list:
                    label_list.append(label2)
                    get_seq_uniprot(label2, seq_uniprot)

                get_dg_uniprot(label1, label2, kd, dg_info_uniprot)


for elem in dg_info_uniprot:
    print(elem, file=open("./pdbbind/pdbbind_uniprot.dg.txt", "a"))


# print(label_list)
for elem in seq_uniprot:
    print(elem, file=open("./pdbbind/pdbbind_uniprot.seq.txt", "a"))

In [95]:
f1.close
f2.close

<function TextIOWrapper.close()>